In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import scipy 
import scipy.io as sio
from  scipy import signal 
from scipy.interpolate import griddata

import copy

In [ ]:
resample_challenge = sio.loadmat('resample_codeChallenge.mat')
resample_challenge

origS = np.squeeze(resample_challenge['origS'])
origT = np.squeeze(resample_challenge['origT'])
signal = np.squeeze(resample_challenge['signal'])
time = np.squeeze(resample_challenge['time'])

print(origS.shape)
print(origT.shape)
print(signal.shape)
print(time.shape)

In [ ]:
plt.plot(time, signal, 'ks-', label='signal')
plt.plot(origT, origS, 'r', label='original')
plt.legend()
#plt.ylim([0, 30])
#plt.xlim([0, 0.1])

plt.show()

# Need to go from signal to original in red
# Get rid of NaNs in signal
# Spectral interpolation to address noise burts
# Upsample to get to regularly spaced data


In [ ]:
plt.plot(np.diff(time))
plt.show()

plt.plot(np.diff(origT))
plt.show()


In [ ]:
# Step 1 find NaNs
np.isnan(signal)
print(sum(np.isnan(signal))) #485
print(len(signal)) # 2399
print(len(origS)) # 3001

foo = len(signal) + sum(np.isnan(signal))
print(foo) # 2884

# Try replacing NaNs with 0
nanIdx = np.isnan(signal)
signalClean = signal[~nanIdx]
timeClean = time[~nanIdx]
plt.plot(timeClean, signalClean, label='nan Removed')
plt.legend()
plt.show



In [ ]:
# Interpolate using original time vector so that it is regularly spaced
newsignal = griddata(timeClean, signalClean, origT, method='linear')
plt.plot(origT, origS, label='original')
plt.plot(origT, newsignal, label='regularly sampled')

plt.legend()
plt.show()

plt.plot(origT, origS, label='original')
plt.plot(origT, newsignal, label='regularly sampled')

plt.legend()
plt.xlim([0.15, 0.5])
plt.ylim([0,25])
plt.show()


In [ ]:
# Identify where noise bursts are and spectrally interpolate between those positions

# Idea take fft to identify noise bursts and filter those out

# make hz 
# FFT
# low-pass filter

dt = np.mean(np.diff(origT))
fs = 1/dt
print(fs) #1000 Hz
npnts = len(newsignal)
print(npnts) #3001

hz_vec = np.linspace(0, fs, npnts)
newsignalX = np.abs(scipy.fft.fft(scipy.signal.detrend(newsignal))/npnts)**2

plt.plot(hz_vec, newsignalX, label='two-sided fft')

hz_vec = np.linspace(0, fs/2, np.int(np.floor((npnts/2)+1)))
plt.plot(hz_vec, newsignalX[0:len(hz_vec)], label='only positive fft')
len(hz_vec)
plt.legend()
plt.xlim([0, 50])
plt.show()

# Low-pass filter the data
lp_cutoff = 12.5
lp_transw = 2
lp_bands = [0, lp_cutoff, lp_cutoff+lp_transw, fs/2]
lp_shape = [1, 1, 0, 0]
lp_order = int(np.round(10*fs/lp_cutoff)+1)

hz_lpfilt = np.linspace(0, fs, lp_order)
print(lp_order)

lp_filter = scipy.signal.firls(lp_order, bands=lp_bands, desired=lp_shape, fs=fs)

# Evaluate filter 
plt.subplot(121)
plt.plot(lp_filter)

plt.subplot(122)
plt.plot(lp_bands,lp_shape,label='ideal')
plt.plot(hz_lpfilt, np.abs(scipy.fft.fft(lp_filter))**2, label='actual')
plt.xlim([0,20])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain')
plt.legend()
plt.show()
# Apply filter
signal_filtered = scipy.signal.filtfilt(lp_filter, 1, newsignal)
signal_filteredX = np.abs(scipy.fft.fft(scipy.signal.detrend(signal_filtered))/npnts)**2

plt.subplot(211)
plt.plot(origT, newsignal, label='signal with high frequency noise bursts')
plt.plot(origT, signal_filtered, label='low-pass filtered signal')
plt.subplot(212)
plt.plot(hz_vec, newsignalX[0:len(hz_vec)], label='only positive fft')
plt.plot(hz_vec, signal_filteredX[0:len(hz_vec)], label='low-pass filtered frequency response')
plt.xlim([0,50])
plt.legend()

In [ ]:
plt.plot(origT, newsignal, label='new signal')
plt.xlim([0.15, 0.5])
plt.ylim([0,25])
plt.show()
# how to find timepoints of noise burst? 
# spectrogram? 

f, t, Sxx = scipy.signal.spectrogram(newsignal, fs = 1000, detrend = 'l', window = 'hann', nperseg=100)

plt.subplot(211)
plt.pcolormesh(t,f,Sxx)
plt.ylim([0,20])


plt.subplot(212)
plt.plot(origT, newsignal)
plt.autoscale(enable=True, axis='x', tight=True)

In [ ]:
len(np.diff(newsignal))
len(origT[1:3001])
len(origT[0:-1])
origT[3000]
origT[-1]
origT[0]
origT[1]
origT[3000]
origT[-1]
print(len(origT[0:3001]))
print(len(origT[0:len(origT)]))
print(len(origT[0:]))

plt.plot(origT[1:], np.diff(newsignal))
#plt.ylim([-30,30])
plt.title('diff of upsampled signal')
plt.show()

plt.plot(np.abs(np.diff(newsignal)) > 24)
plt.title("Noise Burst indices")

In [ ]:
# Low-pass filtering data gets rid of the sharp drops and rises in the original signal
# Try interpolating between noise burst indices instead to get a more faithful recreation of the original data
sig_interp = copy.deepcopy(newsignal)
noise_idx = np.argwhere(np.abs(np.diff(newsignal)>30)) + 3 # adding 1 to account for difference in lengths 
# examine indices for the noise bursts
print(noise_idx)
noise_range = np.append(np.arange(289,311), np.arange(1289,1311))
sig_interp[noise_range] = np.nan

plt.plot(origT, newsignal, label='upsampled')
plt.plot(origT, sig_interp, label='noise burst removed')
plt.legend()
plt.show()

plt.plot(origT, newsignal, label='upsampled')
plt.plot(origT, sig_interp, label='noise burst removed')
plt.legend()
plt.xlim([0.25,0.35])
plt.show()

plt.plot(origT, newsignal, label='upsampled')
plt.plot(origT, sig_interp, label='noise burst removed')
plt.legend()
plt.xlim([1.25,1.35])
plt.show()


In [ ]:
nanIdx = np.isnan(sig_interp)
sig_interpClean = sig_interp[~nanIdx]
sig_timeClean = origT[~nanIdx]

sig_interp_linear = griddata(sig_timeClean, sig_interpClean, origT, method='linear')


In [ ]:

plt.plot(origT, newsignal, label='upsampled')
plt.plot(origT, sig_interp, label='noise burst removed')
plt.plot(origT, sig_interp_linear, label='linear interpolation')
plt.legend()
plt.xlim([0.25,0.35])
plt.show()

plt.plot(origT, newsignal, label='upsampled')
plt.plot(origT, sig_interp, label='noise burst removed')
plt.plot(origT, sig_interp_linear, label='linear interpolation')
plt.legend()
plt.xlim([1.25,1.35])
plt.show()

In [ ]:
plt.plot(origT, origS, label='original signal')
plt.plot(origT, sig_interp_linear, label='cleaned signal')
plt.legend()
plt.xlim([0.5, 1])